# EDA: Diagnosing Diabetes

This [dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database) is from the National Institute of Diabetes and Digestive and Kidney Diseases. It contains the following columns:

- `Pregnancies`: Number of times pregnant
- `Glucose`: Plasma glucose concentration per 2 hours in an oral glucose tolerance test
- `BloodPressure`: Diastolic blood pressure
- `SkinThickness`: Triceps skinfold thickness
- `Insulin`: 2-Hour serum insulin
- `BMI`: Body mass index
- `DiabetesPedigreeFunction`: Diabetes pedigree function
- `Age`: Age (years)
- `Outcome`: Class variable (0 or 1)

## Initial Inspection

In [1]:
import pandas as pd
import numpy as np

# Load data from file
diabetes_data = pd.read_csv('diabetes.csv')
print(diabetes_data.shape)

(768, 9)


This data has 768 rows and 9 columns.

## Further Inspection

In [2]:
# Check for null values
print(diabetes_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    object 
dtypes: float64(2), int64(6), object(1)
memory usage: 54.1+ KB
None


There are no null values.

In [3]:
# Check for anomalous values
print(diabetes_data.describe())

       Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  768.000000     768.000000     768.000000  768.000000   
mean      3.845052  120.894531      69.105469      20.536458   79.799479   
std       3.369578   31.972618      19.355807      15.952218  115.244002   
min       0.000000    0.000000       0.000000       0.000000    0.000000   
25%       1.000000   99.000000      62.000000       0.000000    0.000000   
50%       3.000000  117.000000      72.000000      23.000000   30.500000   
75%       6.000000  140.250000      80.000000      32.000000  127.250000   
max      17.000000  199.000000     122.000000      99.000000  846.000000   

              BMI  DiabetesPedigreeFunction         Age  
count  768.000000                768.000000  768.000000  
mean    31.992578                  0.471876   33.240885  
std      7.884160                  0.331329   11.760232  
min      0.000000                  0.078000   21.000000  
25%     27.300000        

Glucose, blood pressure, skin thickness, insulin and BMI should not be zero, but these columns have at least one value of zero! These values have likely been entered where the data was missing.

17 pregnancies are a lot of pregnancies. Is this value correct? Given the standard deviation, it is an outlier.

A maximum skin thickness of 99 seems very high compared to the third quartile. This data may have been entered incorrectly. The same is true for insulin, BMI, Diabetes Pedigree Function and age.

In [4]:
# Replace zero-values with NaN
diabetes_data[['Glucose',
               'BloodPressure',
               'SkinThickness',
               'Insulin',
               'BMI']] = diabetes_data[['Glucose',
                                        'BloodPressure',
                                        'SkinThickness',
                                        'Insulin',
                                        'BMI']].replace(0, np.NaN)

Check for null values again.

In [5]:
print(diabetes_data.isnull().sum())

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


This is the number of null values for each column. Where do we have missing values?

In [6]:
# Print all rows with missing values
print(diabetes_data[diabetes_data.isnull().any(axis=1)])

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6    148.0           72.0           35.0      NaN  33.6   
1              1     85.0           66.0           29.0      NaN  26.6   
2              8    183.0           64.0            NaN      NaN  23.3   
5              5    116.0           74.0            NaN      NaN  25.6   
7             10    115.0            NaN            NaN      NaN  35.3   
..           ...      ...            ...            ...      ...   ...   
761            9    170.0           74.0           31.0      NaN  44.0   
762            9     89.0           62.0            NaN      NaN  22.5   
764            2    122.0           70.0           27.0      NaN  36.8   
766            1    126.0           60.0            NaN      NaN  30.1   
767            1     93.0           70.0           31.0      NaN  30.4   

     DiabetesPedigreeFunction  Age Outcome  
0                       0.627   50       1  
1                    

All of these rows with missing data seem to have missing data for insulin.

In [7]:
# Filter the data to every row with missing data for insulin
NullInsulin = diabetes_data[diabetes_data['Insulin'].isnull()]
NullInsulin.isnull().sum()

Pregnancies                   0
Glucose                       4
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          10
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [8]:
# Compare to the full set
diabetes_data.isnull().sum() - NullInsulin.isnull().sum()

Pregnancies                 0
Glucose                     1
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         1
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Our conclusion seems to be accurate, except for one row where glucose is missing but insulin is not and one row where BMI is missing, but insulin is not.

Wherever we have missing data, we can be almost certain that insulin is also missing. How are these tests linked?

In [9]:
# Check the data types
diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   763 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   Insulin                   394 non-null    float64
 5   BMI                       757 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    object 
dtypes: float64(6), int64(2), object(1)
memory usage: 54.1+ KB


Why is the `Outcome` column of type `object` (string)?

In [10]:
# print unique values of Outcome column
print(diabetes_data.Outcome.unique())

['1' '0' 'O']


This column is binary, either 1 or 0. Sometimes, the 0 value was entered as a string letter 'O'. We should fix this.

In [16]:
diabetes_data['Outcome'] = diabetes_data['Outcome'].replace('O', 0).astype('int64')

In [18]:
print(diabetes_data.Outcome.unique())

[1 0]


This data has now been inspected, cleaned and prepared for analysis!